# 0STARK Soundness Analysis
`0STARK` is the protocol which is implemented in the [`risc0-zkp`](https://docs.rs/risc0-zkp/latest/risc0_zkp/prove) crate and described in [Scalable Transparent Arguments of RISC-V Integrity](https://dev.risczero.com/proof-system-in-detail.pdf). 

This Python Notebook analyzes the soundness of the 0STARK protocol using four assumption regimes: 

1. Conjectured Security under Best Known Attack
2. Provable Soundness in the List-Decoding Radius
3. Conjectured Soundness using Algebraic Coding Theoretic Conjectures
4. Provable Soundness in the Unique-Decoding Radius

As of this writing, all STARKs used in production rely on Regime 1 in the list above. 
We include analysis in the other three regimes for reference. 

We also have a [soundness calculator built into the `risc0-zkp` crate](https://docs.rs/risc0-zkp/latest/risc0_zkp/prove/soundness/index.html#). RISC Zero's security targets are described in our [Cryptographic Security Model](https://dev.risczero.com/api/security-model). 

We would like to thank: 
- Ben Diamond and Jim Posen (Irreducible) for providing analysis for regimes (2) and (3) in the list above,
- Ying Tong Lai and Nico Mohnblatt (Geometry Research) for their support with translating this analysis into the codebase (see above). 
- Al Kindi (Polygon) for his support in analyzing regime (4). 

In [295]:
import math

k = 2  # negative log of the rate.
rho = 1 / (1 << k)  # rate.
h = 21  # log of the trace length. h = 21 is the largest value accepted by RISC Zero's on-chain verifier contracts.
H = 1 << h  # domain size.
D = H / rho  # domain size, after low-degree extension.
ext_size = 4  # field extension degree.
p = (1 << 31) - (1 << 27) + 1  # field characteristic: 
F = math.pow(p, ext_size)  # extension field size.
num_control = 16 # number of control columns. retrieved from `circuit.get_taps()`, etc. 
num_data = 223 # number of data columns. retrieved from `circuit.get_taps()`, etc. 
num_accum = 40 # number of accum columns. retrieved from `circuit.get_taps()`, etc. 
C = num_control + num_data + num_accum  
L = C + 4  # number of polynomials which ultimately appear in the FRI batch. we need to add 4 extra columns, for the "segment polynomials".
s = 50  # number of FRI queries.
max_combo = 9 # the maximum number of entries from a single column that are referenced in a single constraint. 
max_degree = 5

# Regime 1: Conjectured Security under Best Known Attack

The so-called "Toy Problem Conjecture" from the [ethSTARK](https://eprint.iacr.org/2021/582) paper describes the conjectured security level of the STARK protocol under the best-known attack. We modify this analysis to include an error term associated with the permutation & lookup argument.

### Soundness for permutation/lookup argument
The permutation and lookup argument are both based on grand product accumulators; we can bound `e_permutation` and `e_lookup` by counting the degree of the accumulators and applying the Schwartz-Zippel Lemma. 

Because it makes no difference to the soundness analysis and simplifies the accounting dramatically, we treat these as a single accumulator. We proceed to count the number of terms being accumulated, which gives us the numerator for the Schwartz-Zippel bound. 

We observe that the values being accumulated are stored in the `accum` columns. We find that each row of `accum` contains at most `num_accum / ext_size` elements of Fp4, since each `accum` column holds a base field element.

We find that each Fp4 element in the `accum` section corresponds to at most three terms in the grand product accumulation; `prod = prod * x1 * x2 * x3` is degree 4, and including the mux for the `controlID` brings us to our maximum allowed degree, which is 5. 

In [296]:
accumulator_degree = num_accum / ext_size * (max_degree - 2) * H
e_arguments =  num_accum / ext_size * (max_degree - 2) * H / F
e_FRI_constant = 1 / F
e_FRI_queries = math.pow(rho, s)
e_total_toy_problem = e_FRI_constant + e_FRI_queries + e_arguments

In [297]:
import numpy as np
import pandas as pd

def show_table():
    err_terms = pd.DataFrame(
        [
            ["e_FRI_constant", e_FRI_constant],
            ["e_FRI_queries", e_FRI_queries],
            ["e_arguments", e_arguments],
            ["e_total", e_total_toy_problem],
        ],
        columns=["Term", "P[err]"]
    )
    err_terms["-log_2(P[err])"] = 0 - np.log2(err_terms["P[err]"])
    err_terms = err_terms.drop(columns="P[err]")
    return err_terms
show_table()

,Term,-log_2(P[err])
0,e_FRI_constant,123.627562
1,e_FRI_queries,100.000000
2,e_arguments,97.720672
3,e_total,97.450449


# Regime 2: Provable Soundness in the List-Decoding Radius

*This section was written by Ben Diamond & Jim Posen, based on the description in [Scalable Transparent Arguments of RISC-V Integrity](https://dev.risczero.com/proof-system-in-detail.pdf).*

The parameter `m` below is the "Johnson parameter"; see [[Haböck](https://eprint.iacr.org/2022/1216.pdf)]. This one is subtle, since it doesn't appear directly in the protocol at all; rather, it only appears in the soundness estimates. We have chosen it to here _optimize_ the number of bits of provable security achieved. We comment on this further below.

In [298]:
m = 16
alpha = (1 + 1 / (2 * m)) * math.sqrt(rho)
theta = 1 - alpha

The below error terms reflect the _proven_ / known soundness, and derive from Theorem 1.2 of [[Proximity Gaps](https://eprint.iacr.org/2020/654.pdf)]. The below expressions are based on [[Haböck](https://eprint.iacr.org/2022/1216.pdf)] and [[RISC Zero](https://www.risczero.com/proof-system-in-detail.pdf)], which are in agreement.

In [299]:
e_proximity_gap = math.pow(m + 1/2, 7) / (3 * math.pow(rho, 3/2)) * math.pow(D, 2) / F
e_FRI_constant = (L - 1/2) * e_proximity_gap + (2 * m + 1) * (D + 1) * 16 * math.floor((h + 2) / 4 - 2) / (math.sqrt(rho) * F)
e_FRI_queries = math.pow(1 - theta, s)
e_FRI = e_FRI_constant + e_FRI_queries

In the next couple terms, we'll need the Guruswami–Sudan list-size quantity $L^+$ from [[Haböck](https://eprint.iacr.org/2022/1216.pdf)]. Note that there is a bit of notational confusion here; the RISC Zero spec uses the letter $\mathsf{L}$ to refer to _both_ the number of columns $L$ _and_ to the Guruswami–Sudan list size $L^+$. Of course these are different quantities (with unfortunately similar names). *NOTE: This notational confusion has since been corrected in the RISC Zero spec.*

There's a further subtlety. In order for the proof of [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Theorem 8] to go through, we need to apply FRI with proximity parameter $\theta < 1 - \sqrt{\rho^+}$ (note the +!). In particular, we can't choose `m` _too_ large, or else $\theta := 1 - \sqrt{\rho} \cdot \left( 1 + \frac{1}{2 \cdot m} \right)$ will become larger than $1 - \sqrt{\rho^+}$, and the key step in that theorem will fail. Specifically, the relevant _configuration's_ distance from Reed–Solomon code $\mathsf{RS}_{k^+}(K, D)$ over the function field $K := F(Z)$ will exceed the Johnson bound of that code. For our parameters, this doesn't happen until `m = 131073`, so we have plenty of room.

Separately, when we calculate $L^+$, we need to use a _different_ `m` than the one up above. Specifically, we need to use $m^+$, say, where $m^+$ is chosen sufficiently large that $\theta \leq 1 - \sqrt{\rho^+} \cdot \left( 1 + \frac{1}{2 \cdot m^+} \right)$ holds. This fact is not explicit in [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Theorem 8], and appears to have been an oversight.

Let's calculate a minimal suitable $m^+$:

In [300]:
rho_plus = (H + max_combo) / D  # modified rate of "extension code" in function field; controls the agreement we actually need
assert theta < 1 - math.sqrt(rho_plus)  # make sure `m` isn't too large.
m_plus = math.ceil(1 / (2 * (alpha / math.sqrt(rho_plus) - 1)))
assert theta <= 1 - math.sqrt(rho_plus) * (1 + 1 / (2 * m_plus))
m_plus

17

Thus right now, we only get a difference of 1 between $m$ and $m^+$, but this becomes much more significant as $\theta$ asymptotically approaches $1 - \sqrt{\rho^+}$!

In [301]:
L_plus = (m_plus + 1/2) / math.sqrt(rho_plus)  # note the use of m_plus here.

e_ALI = L_plus * C / F
e_DEEP = L_plus * (4 * (H + max_combo - 1) + (H - 1)) / (F - H - D)
e_PLONK = e * 5 * H / F  # 5 comes from n_{σ_{mem}} == 5.
e_PLOOKUP = e * 15 * H / F  # n_{σ_{bytes}} == 15.

e_total_list_decoding = e_FRI + e_ALI + e_DEEP + e_PLONK + e_PLOOKUP
math.log(e_total_list_decoding, 2)

-39.754114166713755

Thus we appear get around ~40 bits of provable security, overall. Note that our choice `m = 16` actually optimizes this number. Indeed, the results go through for _each possible_ `m`; thus, we are free to choose `m` so as to make the result as good as possible. This is a tradeoff, for various reasons. Indeed, as `m` grows, `e_FRI_constant` gets higher (worse), but `e_FRI_queries` gets lower (better).

The individual values are collected in the table below:

In [302]:
import numpy as np
import pandas as pd

def show_table():
    err_terms = pd.DataFrame(
        [
            ["e_FRI_constant", e_FRI_constant],
            ["e_FRI_queries", e_FRI_queries],
            ["e_ALI", e_ALI],
            ["e_DEEP", e_DEEP],
            ["e_PLONK", e_PLONK],
            ["e_PLOOKUP", e_PLOOKUP],
            ["e_total_list_decoding", e_total_list_decoding],
        ],
        columns=["Term", "P[err]"]
    )
    err_terms["log_2(P[err])"] = np.log2(err_terms["P[err]"])
    err_terms = err_terms.drop(columns="P[err]")
    return err_terms
show_table()

,Term,log_2(P[err])
0,e_FRI_constant,-39.759659
1,e_FRI_queries,-47.780294
2,e_ALI,-110.374161
3,e_DEEP,-95.176350
4,e_PLONK,-98.305634
5,e_PLOOKUP,-96.720672
6,e_total_list_decoding,-39.754114


# Regime 3: Conjectured Soundness using Algebraic Theoretic Conjectures
*This section was written by Ben Diamond & Jim Posen, based on the description in [Scalable Transparent Arguments of RISC-V Integrity](https://dev.risczero.com/proof-system-in-detail.pdf).*

Actually, it's not hard to see that, with the rate `rho = 1/4`, it will not be possible to achieve more than 50 bits of soundness (barring an increase in the number of trials `s = 50`), _even_ assuming Conjecture 8.4 of [[Proximity Gaps](https://eprint.iacr.org/2020/654.pdf)]. Indeed, with these parameters, we have $\log_2 \left( \epsilon_{\text{FRI-queries}} \right) = s \cdot \log_2 \left( \left( 1 + \frac{1}{2 \cdot m} \right) \cdot \frac{1}{2} \right) \geq -s$, where the latter holds even in the limit as $m \rightarrow \infty$. Paradoxically, the larger proximity parameters Conjecture 8.4 makes possible—i.e., in the range $\theta \geq 1 - \sqrt{\rho}$—don't help us here, _unless_ we also make further conjectures about the list-decodability of Reed–Solomon codes up to capacity. Sure enough, the proof of [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Theorem 8] relies on the list-decodability of the Reed–Solomon code $\mathsf{RS}_{k^+}(K, D)$, where $K := F(Z)$ is the infinite function field over $F$ in one indeterminate. This latter list decodability will fail as soon as $\theta \geq 1 - \sqrt{\rho^+}$.

Thus, to make progress, we need to additionally assume [[DEEP-FRI](https://eprint.iacr.org/2019/336.pdf), Conjecture 2.3]. As a digression, we note that this conjecture seems highly unlikely to be proven soon; in fact, our best-known explicit constructions of capacity-achieving codes are far more complicated than standard Reed–Solomon codes; we refer example to [[Guruswami](https://www.nowpublishers.com/article/Details/TCS-007)].

In any case, let's proceed, now assuming [[Proximity Gaps](https://eprint.iacr.org/2020/654.pdf), Conjecture 8.4] _and_ [[DEEP-FRI](https://eprint.iacr.org/2019/336.pdf), Conjecture 2.3]. The latter conjecture in turn depends on an unspecified constant `C_rho`, depending only on `rho`.

In [303]:
c_1 = 1  # first parameter in Proximity Gaps, Conjecture 8.4
c_2 = 1  # second parameter in Proximity Gaps, Conjecture 8.4
C_rho = 1  # unpsecified exponent parameter in DEEP-FRI, Conjecture 2.3

epsilon = 0.05  # also called "eta" in Proximity Gaps
theta = 1 - rho - epsilon  # new proximity parameter! overwrites old one.
assert theta < 1 - rho_plus  # can't exceed the capacity of RS_{k⁺}(K, D).
epsilon_plus = 1 - rho_plus - theta
e_proximity_gap = 1 / math.pow(epsilon * rho, c_1) * L * math.pow(D, c_2) / F  # new value, from Proximity Gaps, Conjecture 8.4
e_FRI_constant = (L - 1/2) * e_proximity_gap + (2 * m + 1) * (D + 1) * 16 * math.floor((h + 2) / 4 - 2) / (math.sqrt(rho) * F)  # add folding error
e_FRI_queries = math.pow(1 - theta, s)  # now using the new, better theta!
e_FRI = e_FRI_constant + e_FRI_queries

L_plus = math.ceil(math.pow(D / epsilon_plus, C_rho))
L_plus

167775761

We note that this is a huge number; indeed, in the setting of [[DEEP-FRI](https://eprint.iacr.org/2019/336.pdf), Conjecture 2.3], the list size depends on the domain size `D`. This wasn't true for the original list size `L_plus` above; indeed, the Guruswami–Sudan list size depends only on the rate and the proximity parameter, but not on the message or block length! As a separate note, since we need to perform list decoding in $\mathsf{RS}_{k^+}(K, D)$, we need to compute the list size $L^+$ based on $\varepsilon^+$, not on $\varepsilon$.

We're implicitly using [[DEEP-FRI](https://eprint.iacr.org/2019/336.pdf), Conjecture 2.3], here, in order for the analogue of [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Thm. 8] to go through. Indeed, [[DEEP-FRI](https://eprint.iacr.org/2019/336.pdf), Conjecture 2.3] here allows us to assert that, even at the proximity parameter $\theta = 1 - \rho - \varepsilon$—as long as $\theta < 1 - \rho^+$ holds—we still get list-decodability in $\mathsf{RS}_{k^+}(K, D)$, where the list size $L^+$, just calculated above, again comes from [[DEEP-FRI](https://eprint.iacr.org/2019/336.pdf), Conjecture 2.3]. Let's proceed:

In [304]:
e_ALI = L_plus * C / F
e_DEEP = L_plus * (4 * (H + max_combo - 1) + (H - 1)) / (F - H - D)
e_PLONK = e * 5 * H / F  # 5 comes from n_{σ_{mem}} == 5.
e_PLOOKUP = e * 15 * H / F  # n_{σ_{bytes}} == 15.

e_total_conjectured_coding_theory = e_FRI + e_ALI + e_DEEP + e_PLONK + e_PLOOKUP
math.log(e_total_conjectured_coding_theory, 2)

-72.94017855099938

Thus we can see that we're getting much better security under the various conjectures. Note that this soundness is very sensitive to $\varepsilon$. (Slimmer $\varepsilon$ means _higher_ $\theta$, and hence better soundness from the queries; but it also means bigger list size, and hence worse soundness for DEEP-ALI.) We have chosen $\varepsilon$ to make this soudness as high as possible.

The soudness is also very sensitive to the unspecified exponents `C_rho`, `c_1` and `c_2` in the conjectures. For example, if we increase `C_rho` to 2, we see that the amount of security we'd get drops to ~59 bits (though can in turn be improved to 60 bits by relaxing $\varepsilon$ to 0.1).

For ease of reading, we've collected all of the relevant soundness levels in the table below.

In [305]:
show_table()

,Term,log_2(P[err])
0,e_FRI_constant,-78.018155
1,e_FRI_queries,-86.848280
2,e_ALI,-88.181482
3,e_DEEP,-72.983671
4,e_PLONK,-98.305634
5,e_PLOOKUP,-96.720672
6,e_total_list_decoding,-39.754114


# Regime 4: Provable Soundness in the Unique-Decoding Radius

*When targeting 100+ bits of provable soundness, our analysis indicates that it may be wise to use analysis in the unique-decoding radius rather than the list-decoding radius. To that end, this section adapts the analysis above to reflect provable soundness in the unique decoding radius. We worked with Al Kindi (Polygon) in order to translate Theorem 8.3 from the [Proximity Gaps] paper into a corresponding result for the unique-decoding radius, finding e_FRI_constant = (D+1) * (L + folding_factor - 1) * num_fri_rounds / F_FRI. An informal version of this analysis is available [here](https://hackmd.io/@pgaf/HkKs_1ytT). A more formal (unpublished) analysis is available upon request. It remains to integrate this analysis into the context of Theorem 8 from [Summary on the FRI Low-Degree Test]. This remaining work is reflected in the TODOs in the calculator.*

TODO - update this section for UDR 
The parameter `m` below is the "Johnson parameter"; see [[Haböck](https://eprint.iacr.org/2022/1216.pdf)]. This one is subtle, since it doesn't appear directly in the protocol at all; rather, it only appears in the soundness estimates. We have chosen it to here _optimize_ the number of bits of provable security achieved. We comment on this further below.

In [306]:
m = 16 # TODO update for UDR... do m and m+ both disappear entirely? 
# i think there will still be some notion of rho+...
alpha = 1 - (1 - rho) / 2 # TODO factcheck whether the "1 - " is correct here
theta = 1 - alpha

e_FRI_constant = (D+1) * (L + (folding_factor - 1) * num_fri_rounds) / F_FRI
e_FRI_queries = math.pow(1 - theta, s)
e_FRI = e_FRI_constant + e_FRI_queries

TODO -- revise this note for unique decoding radius

There's a further subtlety. In order for the proof of [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Theorem 8] to go through, we need to apply FRI with proximity parameter $\theta < 1 - \sqrt{\rho^+}$ (note the +!). In particular, we can't choose `m` _too_ large, or else $\theta := 1 - \sqrt{\rho} \cdot \left( 1 + \frac{1}{2 \cdot m} \right)$ will become larger than $\frac{1 - \rho}{2}$, and the key step in that theorem will fail. Specifically, the relevant _configuration's_ distance from Reed–Solomon code $\mathsf{RS}_{k^+}(K, D)$ over the function field $K := F(Z)$ will exceed the Johnson bound of that code. For our parameters, this doesn't happen until `m = 131073`, so we have plenty of room.

Separately, when we calculate $L^+$, we need to use a _different_ `m` than the one up above. Specifically, we need to use $m^+$, say, where $m^+$ is chosen sufficiently large that $\theta \leq 1 - \sqrt{\rho^+} \cdot \left( 1 + \frac{1}{2 \cdot m^+} \right)$ holds. This fact is not explicit in [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Theorem 8], and appears to have been an oversight.

Let's calculate a minimal suitable $m^+$:

In [307]:
rho_plus = (H + max_combo) / D  # modified rate of "extension code" in function field; controls the agreement we actually need
# todo adjust this assertion for unique decoding radius
assert theta < 1 - math.sqrt(rho_plus)  # make sure `m` isn't too large.
m_plus = math.ceil(1 / (2 * (alpha / math.sqrt(rho_plus) - 1)))
# todo adjust this assertion for unique decoding radius
assert theta <= 1 - math.sqrt(rho_plus) * (1 + 1 / (2 * m_plus))
m_plus

3

In [308]:
L_plus = 1 # Since we restrict our analysis to the unique decoding radius, we set the list-size to be 1.

e_ALI = L_plus * C / F_DEEP
e_DEEP = L_plus * (4 * (H + max_combo - 1) + (H - 1)) / (F_DEEP - H - D)
e_PLONK = ext_size * 5 * H / F_accum  # 5 comes from n_{σ_{mem}} == 5.
e_PLOOKUP = ext_size* 15 * H / F_accum  # n_{σ_{bytes}} == 15.

e_total_unique_decoding = e_FRI + e_ALI + e_DEEP + e_PLONK + e_PLOOKUP
math.log(e_total_unique_decoding, 2)

-78.01498338691894

In [309]:
import numpy as np
import pandas as pd

def show_table():
    err_terms = pd.DataFrame(
        [
            ["e_FRI_constant", e_FRI_constant],
            ["e_FRI_queries", e_FRI_queries],
            ["e_ALI", e_ALI],
            ["e_DEEP", e_DEEP],
            ["e_PLONK", e_PLONK],
            ["e_PLOOKUP", e_PLOOKUP],
        ],
        columns=["Term", "P[err]"]
    )
    err_terms["log_2(P[err])"] = np.log2(err_terms["P[err]"])
    err_terms = err_terms.drop(columns="P[err]")
    return err_terms
show_table()

,Term,log_2(P[err])
0,e_FRI_constant,-78.018155
1,e_FRI_queries,-86.848280
2,e_ALI,-115.503441
3,e_DEEP,-100.305630
4,e_PLONK,-98.305634
5,e_PLOOKUP,-96.720672


# Summary

We close with a summary of all four results.

In [310]:
import numpy as np
import pandas as pd

def show_table():
    err_terms = pd.DataFrame(
        [
            ["Regime 1: Conjectured Soundness (Best Known Attack)", e_total_toy_problem],
            ["Regime 2: Provable Soundness (List-Decoding)", e_total_list_decoding],

            ["Regime 3: Conjectured Soundness (Coding Theoretic)", e_total_conjectured_coding_theory],
            ["Regime 4: Provable Soundness (Unique-Decoding)", e_total_unique_decoding],
        ],
        columns=["Term", "P[err]"]
    )
    err_terms["log_2(P[err])"] = np.log2(err_terms["P[err]"])
    err_terms = err_terms.drop(columns="P[err]")
    return err_terms
show_table()

,Term,log_2(P[err])
0,Regime 1: Conjectured Soundness (Best Known At...,-97.450449
1,Regime 2: Provable Soundness (List-Decoding),-39.754114
2,Regime 3: Conjectured Soundness (Coding Theore...,-72.940179
3,Regime 4: Provable Soundness (Unique-Decoding),-78.014983
